In [ ]:
import torch
from Trainer import Trainer
from Utils import custom_mnist
from Plot_functions import generate_classes, plot_history

In [ ]:
config = dict(
    device="cuda" if torch.cuda.is_available() else "cpu",
    num_classes=10,
    img_size=32,
    channels=1,
    n_epochs=[5,5],
    batch_size=64,
    embedding=100, # latent dimension of embedding
    lr_g=0.0002, # Learning rate for generator
    lr_d=0.0002 # Learning rate for discriminator
)

In [ ]:
experiences = [[0,1,2,3,4],[5,6,7,8,9]]
trainer = Trainer(config=config)
history = trainer.fit_join_retrain(experiences=custom_mnist(experiences = experiences),
                                   buff_img=5000, create_gif=False)

In [ ]:
plot_history(history)

In [ ]:
generate_classes(trainer.generator, config["num_classes"], rows=10, device=config["device"])